# 🧼 Dataset Cleaning – Mexican Labor Market Project

This notebook is dedicated to the cleaning and preparation of microdata files from Banco de México’s Local Labor Market Database. These files contain census-based individual-level information on employment, income, demographics, and other variables from the years 1990 to 2020.

### Objectives:
- Load and inspect `.txt` files containing population and labor data.
- Combine "Informacion Personas" datasets across years into one unified dataset.
- Merge relevant complementary datasets (e.g., income) by `ANIO` and `ID_PERSONA`.
- Prepare the final clean dataset for further analysis in classification, regression, and clustering models.

> ⚠️ **Note**: Since `ID_PERSONA` is not traceable across years, all analysis will treat individuals as part of repeated cross-sectional samples rather than a panel.

In [11]:
import os
from glob import glob
import pandas as pd 
from collections import defaultdict

### 📁 Step 1: Define the Raw Data Directory

We define the path to the `rawdata` folder where all `.txt` files are stored. This will be used to locate and load the sample files.

In [26]:
# Step 1: Set your folder path
project_dir = os.path.dirname("/Users/anahirm/Library/CloudStorage/OneDrive-EcolePolytechnique/Polytecnique/Introduction to Machine Learning/")  # or just use os.getcwd() if you're already in the right place
folder_path = os.path.join(project_dir, "Project-data/", "rawdata")

# Step 2: Find all .txt files in the folder
file_paths = glob(os.path.join(folder_path, '*.txt'))

# Step 3: Read a sample (e.g. 100 rows) from each file
sample_data = {}

for path in file_paths:
    file_name = os.path.basename(path)
    print(f"📄 Reading sample from: {file_name}")
    
    try:
        df_sample = pd.read_csv(path, encoding='latin-1', nrows=800, low_memory=False)
        sample_data[file_name] = df_sample
        print(f"✅ Loaded {df_sample.shape[0]} rows, {df_sample.shape[1]} columns\n")
    except Exception as e:
        print(f"❌ Could not read {file_name}: {e}\n")

📄 Reading sample from: Informacion Personas 2000_0.txt
✅ Loaded 800 rows, 66 columns

📄 Reading sample from: Informacion Personas 2000_1.txt
✅ Loaded 800 rows, 66 columns

📄 Reading sample from: Informacion Personas 2020_0.txt
✅ Loaded 800 rows, 65 columns

📄 Reading sample from: Informacion Personas 2020_1.txt
✅ Loaded 800 rows, 65 columns

📄 Reading sample from: Informacion Derechohabiencia.txt
✅ Loaded 800 rows, 3 columns

📄 Reading sample from: Informacion Personas 1990_0.txt
✅ Loaded 800 rows, 66 columns

📄 Reading sample from: Informacion Prestaciones.txt
✅ Loaded 800 rows, 3 columns

📄 Reading sample from: Informacion Discapacidades.txt
✅ Loaded 800 rows, 4 columns

📄 Reading sample from: Informacion Personas 2010_1.txt
✅ Loaded 800 rows, 65 columns

📄 Reading sample from: Informacion Personas 2010_0.txt
✅ Loaded 800 rows, 65 columns

📄 Reading sample from: Informacion Ingresos.txt
✅ Loaded 800 rows, 4 columns

📄 Reading sample from: Informacion Personas 2015_2.txt
✅ Loaded 800 

### 📊 Step 2: Display Dataset Names and Sample Sizes

We display the names and dimensions (rows and columns) of the datasets that were loaded into memory. This allows us to verify that the sample reading was successful.

In [27]:
# Print the name and size (rows, columns) of each dataset in the dictionary
print("📊 Overview of loaded sample datasets:\n")

for name, df in sample_data.items():
    print(f"{name:<40} →  Rows: {df.shape[0]:>5},  Columns: {df.shape[1]}")

📊 Overview of loaded sample datasets:

Informacion Personas 2000_0.txt          →  Rows:   800,  Columns: 66
Informacion Personas 2000_1.txt          →  Rows:   800,  Columns: 66
Informacion Personas 2020_0.txt          →  Rows:   800,  Columns: 65
Informacion Personas 2020_1.txt          →  Rows:   800,  Columns: 65
Informacion Derechohabiencia.txt         →  Rows:   800,  Columns: 3
Informacion Personas 1990_0.txt          →  Rows:   800,  Columns: 66
Informacion Prestaciones.txt             →  Rows:   800,  Columns: 3
Informacion Discapacidades.txt           →  Rows:   800,  Columns: 4
Informacion Personas 2010_1.txt          →  Rows:   800,  Columns: 65
Informacion Personas 2010_0.txt          →  Rows:   800,  Columns: 65
Informacion Ingresos.txt                 →  Rows:   800,  Columns: 4
Informacion Personas 2015_2.txt          →  Rows:   800,  Columns: 65
Informacion Personas 2015_0.txt          →  Rows:   800,  Columns: 65
Informacion Personas 2015_1.txt          →  Rows:   800

### 🔄 Step 3: Combine All Years into a Single Dataset

We concatenate the "Informacion Personas" datasets from all available years into one combined dataset called `all_personas`. Each row retains its original year through the `ANIO` column.

In [28]:
# Step 1: Filter only "Informacion Personas" files
personas_parts = {
    name: df for name, df in sample_data.items()
    if "Informacion Personas" in name
}

# Step 2: Group files by year extracted from the filename
personas_by_year = defaultdict(list)

for file_name, df in personas_parts.items():
    try:
        year = file_name.split()[2][:4]  # Get year from "Informacion Personas 2015_0.txt"
        personas_by_year[year].append(df)
    except IndexError:
        print(f"⚠️ Could not extract year from: {file_name}")

# Step 3: Concatenate all parts per year
full_personas_by_year = {
    year: pd.concat(dfs, ignore_index=True)
    for year, dfs in personas_by_year.items()
}

# Step 4: Combine all years into one DataFrame
sample_all_personas = pd.concat(
    full_personas_by_year.values(),
    ignore_index=True
)

# Step 5: Print the shape and preview
print(f"🧾 Combined sample dataset: {sample_all_personas.shape[0]:,} rows, {sample_all_personas.shape[1]} columns")
sample_all_personas.head()

🧾 Combined sample dataset: 8,000 rows, 66 columns


,ANIO,ID_PERSONA,LLAVE_ENTIDAD,LLAVE_MUNICIPIO,CLAVE_MUNICIPIO_INEGI(CLAVE_DE_AGEM),LLAVE_LOCALIDAD,CLAVE_LOCALIDAD_INEGI,ID_VIVIENDA,ID_HOGAR,LLAVE_COBERTURA,...,NUMPERSONA,EDAD,ESCOLARIDAD,ESCOLARIDAD_ACUMULADA,INGRESO,HORAS_TRABAJADAS,HIJOS_NACIDOS,HIJOS_VIVOS,HIJOS_FALLECIDOS,MERCADO_TRABAJO_LOCAL
0,2000,200000000000000005147,1,17,1,170001,1.0,2000000000000001,2.000000e+15,0,...,NaN,19.0,6.0,6.0,1929.0,NaN,NaN,NaN,NaN,1
1,2000,200000000000000005154,1,17,1,170001,1.0,2000000000002444,2.000000e+15,0,...,NaN,30.0,5.0,5.0,2143.0,54.0,NaN,NaN,NaN,1
2,2000,200000000000000005162,1,17,1,170001,1.0,2000000000004887,2.000000e+15,0,...,NaN,17.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,1
3,2000,200000000000000005172,1,17,1,170001,1.0,2000000000007330,2.000000e+15,0,...,NaN,18.0,4.0,4.0,2357.0,54.0,NaN,NaN,NaN,1
4,2000,200000000000000005179,1,17,1,170001,1.0,2000000000012214,2.000000e+15,0,...,NaN,24.0,3.0,9.0,1929.0,53.0,NaN,NaN,NaN,1


In [ ]:

# Define the full path to the file
save_path = os.path.join(project_dir, "Project-data", "cleandata", "sample_all_personas.csv")

# Save the DataFrame
#sample_all_personas.to_csv(save_path, index=False, encoding='utf-8')

#print(f"✅ File saved to: {save_path}")

✅ File saved to: /Users/anahirm/Library/CloudStorage/OneDrive-EcolePolytechnique/Polytecnique/Introduction to Machine Learning/Project-data/cleandata/sample_all_personas.csv
